In [6]:
# In[1]:
import time
t1 = time.time()

#importando as bibliotecas 
import torch
import torch_geometric
import numpy as np
import sys 
from torch_geometric.data import Data
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv 
import statistics as sts
import matplotlib.pyplot as plt

sys.path.insert(1, '/home/marina/InCi/')
from cbirdb import acbirdb,cbirdbDistMatrix,evaluator,unsuperalg,recknngraph

execucoes = 10

def DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs): 

    # máscaras 

    train_mask = [] #máscaras de treino - true = treino 
    val_mask = [] #valores das máscaras 
    test_mask = [] #máscaras de testes 
    y = [] # y = vetor de "gabarito" 

    for i in range (pN):# para cada imagem i: 
        y.append(i // classSize) # definindo o vetor de classes 
        d = i % classSize # d = resto do número da imagem dividido pela classe 
        if (d<trPerClass): # se d é menor que o número de imagens de treino por classe, então: 
            valueTr = True # treino = true 
            valueVal = False # teste = false 
        else: # d é maior ou igual ao número de imagens de treino por classe: 
            valueTr = False # treino = false 
            if (d<(valPerClass+trPerClass)): 
                valueVal = True # teste = true 
            else:
                valueVal = False #teste = false 
        valueTest = (not valueTr) and (not valueVal) 

        train_mask.append (valueTr) # criando os vetores de máscaras 
        val_mask.append(valueVal) #vetor de validação 
        test_mask.append(valueTest)

    # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
    train_mask = torch.tensor (train_mask) #torch 
    val_mask = torch.tensor (val_mask)
    test_mask = torch.tensor (test_mask)
    y = torch.tensor (y)

    
    #-------------------------------- Feature Matrix ------------------------------------- 
    #lendo a matriz de features 

    #print ('Reading feature matrix ...')
    feat_matrix_file = "matrix_sem_virgulas.txt" #"feat-matrix.txt"
    x = np.loadtxt(feat_matrix_file, delimiter=" ")
    #print(x.shape)
    x = torch.tensor(x)
    #print (x)
    
    # Indexação das Bordas 

    #arquivo lista 
    pListFile = "list.txt"
    #arquivo dist matrix
    pDSimFile = DistM #"/home/marina/venv/oxford17flowers-caffe-fc7_matrix.txt"


    #print ('Reading DB ...')
    db1 = cbirdbDistMatrix(pListFile,pDSimFile,pN)
    db2 = cbirdbDistMatrix(pListFile,pDSimFile,pN)

    #print ('Making edge list ...')
    edge_index1 = []
    edge_index2 = []

    for qimg in db1.getList(): #para cada imagem da lista 
        # DB ORIGINAL - NO RESTO USAR O DB NOVO 
        #print (qimg)
        crl = db1.getRL(qimg) # mais próximos no ranked list 
        qindex = db1.getIndexOf(qimg) 
        for i in range(pK): # cria arestas entre as k posições mais próximas 
            cindex = db1.getIndexOf (crl[i])
            edge_index1.append([qindex,cindex])
    
    for qimg in db2.getList(): #para cada imagem da lista 
        # DB ORIGINAL - NO RESTO USAR O DB NOVO 
        #print (qimg)
        crl = db2.getRL(qimg) # mais próximos no ranked list 
        qindex = db2.getIndexOf(qimg) 
        for i in range(pK): # cria arestas entre as k posições mais próximas 
            cindex = db2.getIndexOf (crl[i])
            edge_index2.append([qindex,cindex])

    #print(edge_index)

    #passa para o torch 
    # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
    edge_index = torch.tensor(edge_index)
    edge_index = edge_index.t().contiguous()

    #print (edge_index)
    
    #--------------------------------- Data Object  -----------------------------------------
    #data são as informações do grafo completo (eu acho) *_* 

    #print ('Loading data object...')
    data = Data(x=x.float(), edge_index=edge_index, y=y, test_mask=test_mask, train_mask=train_mask,  val_mask=val_mask  )


    #--------------------------------- Graph Neural Network Defiition  -----------------------------------------

    # rede neural: NÃO MEXA!!!!!!!!!!

    #print ('Defining GCN model...')

    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = GCNConv(pNFeatures, pNNeurons) #dataset.num_node_features
            self.conv2 = GCNConv(pNNeurons, numberOfClasses) #dataset.num_classes

        def forward(self, data):
            x, edge_index = data.x, data.edge_index

            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.conv2(x, edge_index)

            return F.log_softmax(x, dim=1)



    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device('cpu')
    model = Net().to(device)
    #data = dataset[0].to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=pLR, weight_decay=5e-4)

    #print ('Training...')

    model.train() # treinando com a rede neural 
    for epoch in range(pNEpochs):	
        #print("Training epoch: ", epoch)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()


    # aqui ele avalia o modelo, com base nas máscaras de teste e treino que definimos no começo     
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / pN #data.test_mask.sum().item() # esse acc é o mesmo da soccer? é um descritor? 

    #print('Accuracy: {:.4f}'.format(acc)) 
    return(acc)


# informações do dataset
pN = 4096 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 300 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos



In [7]:
DistM = "matrix_flowers_rlsim_bonita_corrigida.txt" 
PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP

PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )

0


IndexError: list index out of range

In [ ]:
DistM = "matrix_flowers_rdpac_talvez_bonita_corrigida.txt"
PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP

PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )